<a href="https://colab.research.google.com/github/Abdallah943/pandas_practice/blob/main/data%20analysis%20project%E2%80%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [38]:
# Load all datasets
def load_coffee_data():
    # Load each CSV file
    export = pd.read_csv('Coffee_export.csv')
    production = pd.read_csv('Coffee_production.csv')
    importers = pd.read_csv('Coffee_importers_consumption.csv')
    re_export = pd.read_csv('Coffee_re_export.csv')
    inventory = pd.read_csv('Coffee_green_coffee_inventorie.csv')
    imports = pd.read_csv('Coffee_import.csv')
    domestic = pd.read_csv('Coffee_domestic_consumption.csv')

    return {
        'export': export,
        'production': production,
        'importers': importers,
        're_export': re_export,
        'inventory': inventory,
        'imports': imports,
        'domestic': domestic
    }

In [43]:
# Clean and prepare data
def clean_data(data):
    # Clean production data
    production = data['production'].melt(id_vars=['Country', 'Coffee type'],
                                         var_name='Year',
                                         value_name='Production')
    production['Year'] = production['Year'].str.extract('(\d+)').astype(float)
    production.dropna(subset=['Year'], inplace=True)
    production['Year'] = production['Year'].astype(int)

    # Clean export data
    export = data['export'].melt(id_vars=['Country'],
                                 var_name='Year',
                                 value_name='Export')
    export['Year'] = export['Year'].str.extract('(\d+)').astype(float)
    export.dropna(subset=['Year'], inplace=True)
    export['Year'] = export['Year'].astype(int)

    # Clean domestic consumption data
    domestic = data['domestic'].melt(id_vars=['Country', 'Coffee type'],
                                     var_name='Year',
                                     value_name='Domestic Consumption')
    domestic['Year'] = domestic['Year'].str.extract('(\d+)').astype(float)
    domestic.dropna(subset=['Year'], inplace=True)
    domestic['Year'] = domestic['Year'].astype(int)

    # Merge production and export data to calculate export percentage
    merged_data = pd.merge(production, export, on=['Country', 'Year'], how='left')

    # Calculate Export Percentage, handling potential division by zero and NaNs
    merged_data['Export Percentage'] = (merged_data['Export'] / merged_data['Production']) * 100
    merged_data['Export Percentage'] = merged_data['Export Percentage'].replace([np.inf, -np.inf], np.nan)


    return {'production': production, 'export': export, 'domestic': domestic, 'merged_data': merged_data}

In [17]:
# Analysis 1: Top Coffee Producing Countries
def top_producers(df, n=10):
    top = df.groupby('Country')['Production'].sum().nlargest(n).reset_index()
    fig = px.bar(top, x='Production', y='Country', orientation='h',
                 title=f'Top {n} Coffee Producing Countries (1990-2019)',
                 labels={'Production': 'Total Production (tons)'})
    fig.update_layout(yaxis={'categoryorder':'total ascending'})
    return fig

In [18]:
# Analysis 2: Production Trends Over Time
def production_trends(df):
    trend = df.groupby('Year')['Production'].sum().reset_index()
    fig = px.line(trend, x='Year', y='Production',
                  title='Global Coffee Production Trend (1990-2019)',
                  labels={'Production': 'Total Production (tons)'})
    return fig

In [19]:
# Analysis 3: Production by Coffee Type
def production_by_type(df):
    by_type = df.groupby(['Year', 'Coffee type'])['Production'].sum().reset_index()
    fig = px.area(by_type, x='Year', y='Production', color='Coffee type',
                  title='Coffee Production by Type (1990-2019)',
                  labels={'Production': 'Total Production (tons)'})
    return fig


In [20]:
# Analysis 4: Domestic Consumption vs Production
def consumption_vs_production(df):
    # Get latest year data for each country
    latest = df[df['Year'] == df['Year'].max()]
    fig = px.scatter(latest, x='Production', y='Domestic Consumption',
                     color='Coffee type', hover_name='Country',
                     title='Domestic Consumption vs Production (2019)',
                     labels={'Production': 'Production (tons)',
                             'Domestic Consumption': 'Domestic Consumption (tons)'})
    fig.add_shape(type="line", x0=0, y0=0, x1=1, y1=1,
                  xref="paper", yref="paper",
                  line=dict(color="gray", width=2, dash="dot"))
    return fig


In [21]:
# Analysis 5: Export Percentage by Country
def export_percentage(df):
    avg_export = df.groupby('Country')['Export Percentage'].mean().reset_index()
    avg_export = avg_export[avg_export['Export Percentage'] > 0].nlargest(15, 'Export Percentage')
    fig = px.bar(avg_export, x='Export Percentage', y='Country', orientation='h',
                 title='Average Export Percentage of Production (Top 15 Countries)',
                 labels={'Export Percentage': 'Percentage of Production Exported'})
    fig.update_layout(yaxis={'categoryorder':'total ascending'})
    return fig


In [49]:
# Generate all visualizations
top_producers_fig = top_producers(cleaned_data['production'])
production_trends_fig = production_trends(cleaned_data['production'])
production_by_type_fig = production_by_type(cleaned_data['production'])

# Merge production and domestic consumption data for consumption vs production analysis
consumption_data = pd.merge(cleaned_data['production'], cleaned_data['domestic'], on=['Country', 'Year', 'Coffee type'], how='inner')

consumption_vs_production_fig = consumption_vs_production(consumption_data)

# The export percentage analysis likely needs more data than currently returned by clean_data
# For now, I will comment it out and address it separately.
export_percentage_fig = export_percentage(cleaned_data['merged_data'].dropna(subset=['Export Percentage']))

In [45]:
# Load and clean data
data = load_coffee_data()
cleaned_data = clean_data(data)

# Display the keys of the cleaned_data dictionary
print("Keys in cleaned_data:", cleaned_data.keys())

# Display the first few rows of the cleaned production and export dataframes
display(cleaned_data['production'].head())
display(cleaned_data['export'].head())
display(cleaned_data['domestic'].head()) # Display domestic data as well
display(cleaned_data['merged_data'].head()) # Display merged_data as well

Keys in cleaned_data: dict_keys(['production', 'export', 'domestic', 'merged_data'])


,Country,Coffee type,Year,Production
0,Angola,Robusta/Arabica,1990,3.000000e+06
1,Bolivia (Plurinational State of),Arabica,1990,7.380000e+06
2,Brazil,Arabica/Robusta,1990,1.637160e+09
3,Burundi,Arabica/Robusta,1990,2.922000e+07
4,Ecuador,Arabica/Robusta,1990,9.024000e+07


,Country,Year,Export
0,Angola,1990,5040000
1,Bolivia (Plurinational State of),1990,9360000
2,Brazil,1990,1016160000
3,Burundi,1990,35100000
4,Cameroon,1990,156660000


,Country,Coffee type,Year,Domestic Consumption
0,Angola,Robusta/Arabica,1990,1200000
1,Bolivia (Plurinational State of),Arabica,1990,1500000
2,Brazil,Arabica/Robusta,1990,492000000
3,Burundi,Arabica/Robusta,1990,120000
4,Ecuador,Arabica/Robusta,1990,21000000


,Country,Coffee type,Year,Production,Export,Export Percentage
0,Angola,Robusta/Arabica,1990,3.000000e+06,5040000,168.000000
1,Bolivia (Plurinational State of),Arabica,1990,7.380000e+06,9360000,126.829268
2,Brazil,Arabica/Robusta,1990,1.637160e+09,1016160000,62.068460
3,Burundi,Arabica/Robusta,1990,2.922000e+07,35100000,120.123203
4,Ecuador,Arabica/Robusta,1990,9.024000e+07,107040000,118.617021


In [32]:
# Show all figures
top_producers_fig.show()


In [33]:
production_trends_fig.show()


In [34]:
production_by_type_fig.show()


In [41]:
consumption_vs_production_fig.show()


In [50]:
export_percentage_fig.show()

In [55]:
# Additional analysis for trade data
def analyze_trade(data):
    # Clean import data
    imports = data['imports']
    imports = imports.melt(id_vars=['Country'],
                          var_name='Year',
                          value_name='Import')
    imports['Year'] = imports['Year'].str.extract('(\d+)').astype(float) # Convert to float first to handle NaNs
    imports.dropna(subset=['Year'], inplace=True) # Drop rows where Year is NaN
    imports['Year'] = imports['Year'].astype(int) # Now convert to int

    # Clean re-export data
    re_export = data['re_export']
    re_export = re_export.melt(id_vars=['Country'],
                               var_name='Year',
                               value_name='Re-export')
    re_export['Year'] = re_export['Year'].str.extract('(\d+)').astype(float) # Convert to float first to handle NaNs
    re_export.dropna(subset=['Year'], inplace=True) # Drop rows where Year is NaN
    re_export['Year'] = re_export['Year'].astype(int) # Now convert to int

    # Merge trade data
    trade = pd.merge(imports, re_export, on=['Country', 'Year'], how='outer')
    trade['Net Import'] = trade['Import'] - trade['Re-export']

    # Top importers
    top_importers = trade.groupby('Country')['Import'].sum().nlargest(10).reset_index()
    fig1 = px.bar(top_importers, x='Import', y='Country', orientation='h',
                  title='Top 10 Coffee Importing Countries (1990-2019)',
                  labels={'Import': 'Total Imports (tons)'})
    fig1.update_layout(yaxis={'categoryorder':'total ascending'})

    # Import trends
    import_trend = trade.groupby('Year')['Import'].sum().reset_index()
    fig2 = px.line(import_trend, x='Year', y='Import',
                   title='Global Coffee Import Trend (1990-2019)',
                   labels={'Import': 'Total Imports (tons)'})

    return fig1, fig2

trade_fig1, trade_fig2 = analyze_trade(coffee_data)
trade_fig1.show()
trade_fig2.show()